In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.max_columns', None)

In [ ]:
# deepseek
from openai import OpenAI
client = OpenAI(api_key="", base_url="https://api.deepseek.com")

#### Normaliza JSON

In [ ]:
with open(r'C:\Users\Natan\Documents\FIAP\Modulo 5\Dados\prospects\prospects.json', "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
df = pd.json_normalize(
    data.values(),
    record_path=['prospects'],
    meta=['titulo']
)

In [ ]:
df

,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,titulo
0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,CONSULTOR CONTROL M
1,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , não tem empresa aberta",Ana Lívia Moreira,CONSULTOR CONTROL M
2,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Domain Consultant
3,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Domain Consultant
4,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,2021-2605708-Microfocus Application Life Cycle Management - HP Quality Center-Engineering Lead
...,...,...,...,...,...,...,...,...
53754,Marcelo Azevedo,2018,Encaminhado ao Requisitante,26-02-2025,26-02-2025,,Yasmin da Rosa,SAP SD
53755,Dr. Vitor Hugo Silva,40384,Entrevista Técnica,26-02-2025,27-02-2025,"Entrevista agendada para 27/02, às 9:30",Laura Pacheco,Java Support Analyst
53756,Ana Cardoso,16828,Desistiu,26-02-2025,28-02-2025,Recebeu a confirmação de outro processo seletivo (remoto) e aceitou.,Elisa Nunes,Consultor Sênior Especialista SAP LES-TRA - 1433
53757,Pedro Lucas das Neves,15042,Encaminhado ao Requisitante,28-02-2025,28-02-2025,,Elisa Nunes,Consultor Sênior Especialista SAP LES-TRA - 1433


#### Filtra situacoes aprovadas

In [ ]:
df['situacao_candidado'].unique()

array(['Encaminhado ao Requisitante', 'Contratado pela Decision',
       'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente',
       'Prospect', 'Não Aprovado pelo RH', 'Aprovado',
       'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Contratado como Hunting',
       'Desistiu da Contratação', 'Entrevista com Cliente',
       'Documentação CLT', 'Recusado', 'Documentação Cooperado',
       'Sem interesse nesta vaga', 'Encaminhar Proposta',
       'Proposta Aceita'], dtype=object)

In [ ]:
status_ok = [
    'Contratado pela Decision',
    'Aprovado',
    'Contratado como Hunting',
    'Proposta Aceita'
]

In [ ]:
df_aprovados = df[df['situacao_candidado'].isin(status_ok)]
df_aprovados = df_aprovados[['titulo', 'codigo']]
df_aprovados

,titulo,codigo
2,2021-2607395-PeopleSoft Application Engine-Domain Consultant,25364
4,2021-2605708-Microfocus Application Life Cycle Management - HP Quality Center-Engineering Lead,26338
16,Microfocus QTP - UFT Automation Testing-Lead-2021-2608031,26338
22,Microfocus QTP - UFT Automation Testing-Lead-2021-2608031,22977
38,Datastage - 11739197,19972
...,...,...
53436,Analista de Business Intelligence (BI),49878
53526,Gerente de Projetos Sênior,49887
53529,Consultor SAP SD - FSR 4449,32530
53589,Analista de Dados ETL Sênior,49917


#### Trata arquivo de aplicacoes

In [ ]:
df = pd.read_json(r'C:\Users\Natan\Documents\FIAP\Modulo 5\Dados\applicants\applicants.json')

In [ ]:
with open(r'C:\Users\Natan\Documents\FIAP\Modulo 5\Dados\applicants\applicants.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
df = pd.json_normalize(data.values(), sep='.')

In [ ]:
codigos_aprovados = df_aprovados['codigo'].unique().tolist()
len(codigos_aprovados)

2991

In [ ]:
df_aprovados_vagas = df[df['infos_basicas.codigo_profissional'].isin(codigos_aprovados)][['infos_basicas.codigo_profissional','cv_pt']]
df_aprovados_vagas = df_aprovados_vagas.rename(columns={
    'infos_basicas.codigo_profissional': 'codigo'
})

In [ ]:
print(df_aprovados_vagas['cv_pt'].head(1))

10    \n\nanalista programador | cobol | mainframe\n\nanalista programador com extensa experiência em ambiente mainframe utilizando linguagem cobol, easytreave e natural(online/batch). ao longo de 33 anos apliquei meus conhecimentos de análise e desenvolvimento de sistemas contábeis, financeiros, pagamentos eletrônicos, corporativos, etc.\n\nqualificações\n\nª experiência em desenvolvimento de sistemas administrativos (e.r.p.), corporativos e operacionais (c.r.m.).\n\nª reengenharia, implementação de sistemas, desenvolvimento de projeto físico e lógico, modelagem e levantamento de dados. modelagem e mapeamento funcional, definição e mapeamento de instâncias (teste unitário/teste integrado/simulação e homologação). leitura e definição do escopo do projeto/sistema. definição, desenho e desenvolvimento de sistemas multiplataforma (sistemas integrados, mainframe/web servisse, java). participação ativa em definições de (método e política de promoção de módulos, sla de atendimento às ocorren

In [ ]:
df_aprovados

,titulo,codigo
2,2021-2607395-PeopleSoft Application Engine-Domain Consultant,25364
4,2021-2605708-Microfocus Application Life Cycle Management - HP Quality Center-Engineering Lead,26338
16,Microfocus QTP - UFT Automation Testing-Lead-2021-2608031,26338
22,Microfocus QTP - UFT Automation Testing-Lead-2021-2608031,22977
38,Datastage - 11739197,19972
...,...,...
53436,Analista de Business Intelligence (BI),49878
53526,Gerente de Projetos Sênior,49887
53529,Consultor SAP SD - FSR 4449,32530
53589,Analista de Dados ETL Sênior,49917


In [ ]:
df_resultante = df_aprovados.merge(df_aprovados_vagas, on='codigo')

In [ ]:
df_resultante = df_resultante[df_resultante['cv_pt'] != '']

In [ ]:
df_resultante.head(1)

titulo codigo  \
0  2021-2607395-PeopleSoft Application Engine-Domain Consultant  25364   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

#### DeepSeek: Manipulacao de texto

In [ ]:
# funcao que realiza chamada da API deepseek
def call_deepseek(cv_pt):    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": """
    Por favor extraia informações relevantes do texto abaixo:
    - Habilidades / competências
    - Anos de experiência
    - Idiomas
    - Cargos / profissões
    - Formação acadêmica
    - Empresas anteriores

    PS: Não inclua frases extras, me retorna apenas as descrições requisitadas. Não gera lista de tópicos, evite pular linhas.
    *** Texto:
    """+cv_pt+"""   
    """},
        ],
        stream=False
    )

    return response.choices[0].message.content    

In [ ]:
# Paralelizacao da chamada
responses = [None] * len(df_resultante)
MAX_WORKERS = 20

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as pool:
    
    future_to_idx = {
        pool.submit(call_deepseek, row["cv_pt"]): idx
        for idx, row in df_resultante.iterrows()
    }
    
    for fut in as_completed(future_to_idx):
        idx = future_to_idx[fut]
        try:
            responses[idx] = fut.result()
        except Exception as e:
            responses[idx] = f"ERROR: {e}"

df_resultante["response"] = responses

In [ ]:
df_resultante = df_resultante[['titulo', 'response']]
df_resultante

,titulo,response
0,2021-2607395-PeopleSoft Application Engine-Domain Consultant,"Habilidades / competências: liderança técnica, gerenciamento de projetos, consultoria, desenvolvimento e implantação de sistemas ERP, CRM, HCM, Peoplesoft, análise de requisitos, administração de acessos, suporte a produção, treinamento, integração de sistemas, PL/SQL, Oracle OIC, Microsoft SQL Server, MySQL, Oracle, JavaScript, HTML, CSS, jQuery, Python. Anos de experiência: 30 anos na área de TI. Idiomas: inglês avançado, espanhol intermediário, português nativo. Cargos / profissões: líder de consultoria, gerente de projeto, líder técnico, analista sênior, consultor pleno-sênior. Formação acadêmica: Tecnólogo em Processamento de Dados - Universidade Castelo Branco (1995). Empresas anteriores: DBS-Digital, ATAWAY, Grupo Quanam, Oracle do Brasil, Stefanini Consultoria, F2C (Hitachi Consulting), Infosys, Limine Solutions, Brazilcap, Ogeda IT Solutions, Atos Origin/Peoplefriend, Adoption/HG Global, HQS Consulting."
1,2021-2605708-Microfocus Application Life Cycle Management - HP Quality Center-Engineering Lead,"- Habilidades / competências: desenvolvimento de testes automatizados com java e selenium, cypress, jenkins, ruby, calabash, capybara, appium, cucumber, vb.net, hp uft, silk test, oracle, sql, sqlite, git, maven, ibm rqm, ibm rtc, springboot, sap, soapui, jmeter, plsql, power center, sharepoint, .net, osb, bpel, splunk, tfs, test link, mantis, hpqc, eclipse, selenium ide, selenium webdriver, cucumber, sybase, xml, web service, erp. \n- Anos de experiência: desde 2012 até presente. \n- Idiomas: inglês intermediário, espanhol intermediário. \n- Cargos / profissões: líder de equipes de testes automatizados, scrum master, consultor técnico, analista de qualidade, gerente de banco de dados, implantador e consultor de sistema erp. \n- Formação acadêmica: superior em análise e desenvolvimento de sistemas (Faculdade Anhanguera) 4º, 2º grau técnico em informática (Colégio Brasileiro). \n- Empresas anteriores: Wipro Solutions, Cognizant, Sigga, Take, 123milhas, Sysmap Solutions, Bionexo do Brasil, Colpany Temper, Sync Softwares."
2,Microfocus QTP - UFT Automation Testing-Lead-2021-2608031,"- Habilidades / competências: desenvolvimento de testes automatizados com java, selenium, cypress; liderança de equipes; scrum master; mobile farm local; devicefarm aws; gerenciamento de banco de dados; metodologias ágeis; ruby, calabash, capybara, appium, cucumber, vb.net, hp uft, silk test, oracle, sql, sqlite, git, maven, ibm rqm, ibm rtc, springboot, soapui, jmeter, testrail, selenium ide, eclipse, intellij. \n- Anos de experiência: desde 2012 até presente (2023). \n- Idiomas: inglês intermediário, espanhol intermediário. \n- Cargos / profissões: líder de equipes de testes automatizados, scrum master, consultor técnico, analista de qualidade, gerente de projetos, implantador de sistemas erp. \n- Formação acadêmica: superior em análise e desenvolvimento de sistemas (Faculdade Anhanguera), 2º grau técnico em informática (Colégio Brasileiro). \n- Empresas anteriores: Wipro Solutions (2021-presente), Cognizant (2020-2021), Sigga (2019-2020), Take (2019), 123milhas (2018-2019), Sysmap Solutions (2015-2018), Bionexo do Brasil (2013-2015), Colpany Temper (2013), Sync Softwares (2012-2013)."
3,Datastage - 11739197,"Habilidades / competências: coordenação de projetos, administrador de rede, analista/programador, analista de sistemas, administração de dados, modelagem de dados, projetos de BI/DW, especificação, modelagem de dados, administração de dados, desenvolvimento, validações, construção, definição de melhores práticas, gerenciamento de projetos, liderança técnica. Anos de experiência: 18 anos. Idiomas: inglês avançado. Cargos / profissões: consultor de informática, consultor BI sênior, analista de sistemas, coordenador de projetos. Formação acadêmica: pós-graduação em Master in Business Intelligence – FASP 2008, superior em Tecnologia de Processamento de Dados - Fac

In [ ]:
df_resultante['contexto'] = df_resultante['titulo'] + ' ' + df_resultante['response']

In [ ]:
df_resultante['contexto'].to_csv('candidatos_aprovados.csv', index=False, sep=';')